<a href="https://colab.research.google.com/github/KiaroRB/data-analysis-projects/blob/main/bronze_data_ingestion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
      .appName("bronze_ingestion")\
      .getOrCreate()

In [3]:
!pip install -q gdown

In [7]:
import gdown

file_id = "1LHeH732nph-c0_rrkMJlXuX0xKCqkKHz"
gdown.download(id=file_id, output="online_retail.csv", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1LHeH732nph-c0_rrkMJlXuX0xKCqkKHz
To: /content/online_retail.csv
100%|██████████| 45.6M/45.6M [00:01<00:00, 37.9MB/s]


'online_retail.csv'

In [8]:
df = spark.read.csv(
    "online_retail.csv",
    header=True,
    inferSchema=True
)